Goal of analysis script is to pull in txt files of annotated USVs and prepare to be used to find corresponding spectrogram slices from wav files

Import modules

In [1]:
import pandas as pd
import numpy as np
from tkinter import filedialog
from tkinter import *
import re
import os
from scipy import stats

# visualization
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

Open dialog box and prompt user to select directory containing the data files 

In [3]:
root = Tk()
root.directory = filedialog.askdirectory()
print (root.directory)

C:/Users/Schindler/Documents/Schindler_Lab/Data/USVs/CPA_pair_exp/CPA_pair_tables


Create a path name for each data file

In [4]:
path_names = []
files = os.listdir(root.directory)
for file in files: 
        path_names.append(root.directory + "/" + file)

path_names[0]

'C:/Users/Schindler/Documents/Schindler_Lab/Data/USVs/CPA_pair_exp/CPA_pair_tables/533.Table.1.selections.CPApair.txt'

Read in and create dataframe of animal parameters (pair: 1 = neutral, 2 = CPA pair; Group: 4 = sham, 5 = blast)

In [5]:
group_file = "C:/Users/Schindler/Documents/Schindler_Lab/Data/Analysis/Excel files/USV/3x_groups.xlsx"
data = pd.read_excel(group_file)
df_mice = pd.DataFrame(data = data)
df_mice = df_mice.drop([15], axis = 0)
df_mice.head()

,Animal,Pair,Group,Notes
0,527,1,5,weak blast; scruffy
1,529,1,5,NaN
2,533,2,5,cage flood am of pair
3,534,2,5,cage flood am of pair
4,535,2,5,cage flood am of pair


Loop through path names to create pandas data table from excel file. Get animal number from path name (animal number is part of file name), use animal number to get group and pair codes from the df_mice table, create array of animal number, group and pair codes the length of the data_table and add each as a column.

In [6]:
data_table = pd.DataFrame()

i = 0

for path in path_names:
    data = pd.read_table(path)
    data_table_int = pd.DataFrame(data = data)
    
    name = re.search("\d\d\d", path).group(0)
    name_mult = [name] * len(data_table_int)
    session = re.search("base|cagepair|CPApair", path).group(0)
    session = [session] * len(data_table_int)
    
    group = df_mice[df_mice['Animal'] == int(name)]['Group'].values[0]
    group = [group] * len(data_table_int)
    pair = df_mice[df_mice['Animal'] == int(name)]['Pair'].values[0]
    pair = [pair] * len(data_table_int)
    i = i + 1
    
    data_table_int = data_table_int.assign(Animal = name_mult)
    data_table_int = data_table_int.assign(Group = group)  
    data_table_int = data_table_int.assign(Pair = pair)
    data_table_int = data_table_int.assign(Session = session)
    
    data_table = data_table.append(data_table_int, ignore_index=True)

data_table.head()

,Selection,View,Channel,Begin Time (s),End Time (s),Low Freq (Hz),High Freq (Hz),Delta Time (s),Delta Freq (Hz),Avg Power Density (dB FS),Annotation,Animal,Group,Pair,Session
0,175,Spectrogram 1,1,376.574455,376.583963,7191.2,9940.8,0.010,2749.6,-32.3,low slug,533,5,2,CPApair
1,2,Spectrogram 1,1,46.306579,46.316881,12901.9,14805.4,0.010,1903.5,-41.9,low slug,533,5,2,CPApair
2,13,Spectrogram 1,1,243.272865,243.279204,12901.9,15016.9,0.006,2115.0,-41.2,low slug,533,5,2,CPApair
3,8,Spectrogram 1,1,149.708324,149.714663,14170.9,16074.5,0.006,1903.6,-39.8,low slug,533,5,2,CPApair
4,12,Spectrogram 1,1,243.176192,243.186493,13324.9,16074.5,0.010,2749.6,-41.5,low slug,533,5,2,CPApair


In [7]:
data_table.drop(['Selection', 'View', 'Channel'], axis=1, inplace = True)

In [8]:
data_table.head()

,Begin Time (s),End Time (s),Low Freq (Hz),High Freq (Hz),Delta Time (s),Delta Freq (Hz),Avg Power Density (dB FS),Annotation,Animal,Group,Pair,Session
0,376.574455,376.583963,7191.2,9940.8,0.010,2749.6,-32.3,low slug,533,5,2,CPApair
1,46.306579,46.316881,12901.9,14805.4,0.010,1903.5,-41.9,low slug,533,5,2,CPApair
2,243.272865,243.279204,12901.9,15016.9,0.006,2115.0,-41.2,low slug,533,5,2,CPApair
3,149.708324,149.714663,14170.9,16074.5,0.006,1903.6,-39.8,low slug,533,5,2,CPApair
4,243.176192,243.186493,13324.9,16074.5,0.010,2749.6,-41.5,low slug,533,5,2,CPApair


In [9]:
annot_info = data_table[['Begin Time (s)', 'Annotation', 'Animal', 'Group']]
annot_info.head()

,Begin Time (s),Annotation,Animal,Group
0,376.574455,low slug,533,5
1,46.306579,low slug,533,5
2,243.272865,low slug,533,5
3,149.708324,low slug,533,5
4,243.176192,low slug,533,5


In [12]:
annot_info['Begin Time (s)_1000'] = annot_info['Begin Time (s)']*1000
annot_time_stamps = []
values = annot_info['Begin Time (s)_1000'].values
for value in values:
    time_stamp_num = int(value / 22.5)
    time_stamp_index = time_stamp_num*22.5
    annot_time_stamps.append(time_stamp_index)

annot_info['time_stamp'] = annot_time_stamps
print(annot_info.shape)
annot_info.head()

(552, 6)


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,Begin Time (s),Annotation,Animal,Group,Begin Time (s)_1000,time_stamp
0,376.574455,low slug,533,5,376574.45450,376560.0
1,46.306579,low slug,533,5,46306.57941,46305.0
2,243.272865,low slug,533,5,243272.86510,243270.0
3,149.708324,low slug,533,5,149708.32400,149692.5
4,243.176192,low slug,533,5,243176.19170,243157.5


In [13]:
annot_info.replace('mid slug', 'low slug', inplace = True)
annot_info.replace(['bbc wide', 'bbc long', 'bbc high', 'bbc low'], 'bbc')
annot_info['Annotation'].unique()

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3798: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  method=method)


array(['low slug', 'radar', 'low multi', 'high slug', 'high multi', 'bbc',
       'noise'], dtype=object)

In [14]:
writer = pd.ExcelWriter('annot_info_df.xlsx', engine='xlsxwriter')
annot_info.to_excel(writer, sheet_name='Sheet1')
writer.save()